__Note__: Данные вытянул руками, потому что было плохо

In [1]:
# ! python data/prepare_lsun.py data/image/cat data/image/lmdb/cat

^C


In [21]:
! bash ./data/download_sketch_data.sh

--2022-03-24 00:39:54--  https://www.cs.cmu.edu/~gansketching/files/sketch_data.zip
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218420 (2,1M) [application/zip]
Saving to: ‘./data/sketch_data.zip’

./data/sketch_data. 100%[===================>]   2,12M   163KB/s    in 18s     

2022-03-24 00:40:12 (123 KB/s) - ‘./data/sketch_data.zip’ saved [2218420/2218420]

Archive:  ./data/sketch_data.zip
   creating: ./data/sketch/
   creating: ./data/sketch/photosketch/
   creating: ./data/sketch/photosketch/standing_cat/
 extracting: ./data/sketch/photosketch/standing_cat/671186.png  
 extracting: ./data/sketch/photosketch/standing_cat/160659.png  
 extracting: ./data/sketch/photosketch/standing_cat/397802.png  
 extracting: ./data/sketch/photosketch/standing_cat/454577.png  
 extracting: ./data/sketch/photosketch/standing_cat/352521.png  
 extracting: .

__Note__: чиним зависимости тут+в конде+в терминале

In [6]:
! pip install ninja

In [11]:
import os
print(os.environ.get('CUDA_HOME'))
os.environ["CUDA_HOME"] = '/usr/local/cuda-11.6/'


/usr/local/cuda-11.6/


In [4]:
import torch

torch.cuda.is_available()

True

In [20]:
! ls ./data/

download_eval_data.sh  download_sketch_data.sh	prepare_lsun.py
download_lsun.sh       image


__Note__: тренируем всю модель с нуля

In [22]:
! python train.py \
    --name teaser_cat_augment --batch 4 \
    --dataroot_sketch ./data/sketch/by_author/cat \
    --dataroot_image ./data/image/cat --l_image 0.7 \
    --g_pretrained ./pretrained/stylegan2-cat/netG.pth \
    --d_pretrained ./pretrained/stylegan2-cat/netD.pth \
    --max_iter 150000 --disable_eval --diffaug_policy translation


Using pretrained weight for D1...
Using pretrained weight for D_image...
----------------- Options ---------------
                    batch: 4                             
                    beta1: 0.0                           
                    beta2: 0.99                          
       channel_multiplier: 2                             
          checkpoints_dir: checkpoint                    
             d_pretrained: ./pretrained/stylegan2-cat/netD.pth	[default: ]
              d_reg_every: 16                            
           dataroot_image: ./data/image/cat              	[default: None]
          dataroot_sketch: ./data/sketch/by_author/cat   	[default: None]
           diffaug_policy: translation                   	[default: ]
             disable_eval: True                          	[default: False]
             display_freq: 2500                          
          display_winsize: 400                           
      dsketch_no_pretrain: False                     

__Note__: в таком формате училось в Датасфере

In [ ]:
cd GANSketching && python3 train.py \
    --name quickdraw_single_cat2_augment --batch 1 \
    --dataroot_sketch ./data/sketch/quickdraw/single_sketch/cat02 \
    --dataroot_image ./data/image/cat --l_image 0.7 \
    --g_pretrained ./pretrained/stylegan2-cat/netG.pth \
    --d_pretrained ./pretrained/stylegan2-cat/netD.pth \
    --max_iter 71000 --disable_eval --diffaug_policy translation \

Логгинг в тетрадке ручками)

Считаем метрики

(my_full) {'fid': 49.67758890025698}
(0) {'fid': 84.177413750972}
(1) {'fid': 75.5782804149544}
(2) {'fid': 109.80428746696228}
(3) {'fid': 68.56169285234478}
(4) {'fid': 90.60169181476311}

In [86]:
! python run_metrics.py --models_list weights/eval_list --output metric_results.csv


/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
2
Found 2500 images in the folder ./data/eval/standing_cat/image
FID image : 100%|███████████████████████████████| 79/79 [00:29<00:00,  2.66it/s]
Found 2500 images in the folder ./cache_files/1/image
FID image : 100%|███████████████████████████████| 79/79 [00:33<00:00,  2.38it/s]
(1) {'fid': 75.5782804149544}
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
2
Found 2500 images in the folder ./data/eval/standing_cat/image
FID image : 100%|███████████████████████████████| 79/79 [00:37<00:00,  2.09it/s]
Found 2500 images in the folder ./cache_files/2/image
FID image : 100%|████████████████

__Интерполяция в ganspace__: подбирались id компоненты, слои и скаляр руками

In [59]:
! python ganspace.py --obj cat --comp_id 27 --scalar 50 --layers 2,4 --ckpt weights/my_full.pth --save_dir output/ganspace_fur_standing_cat

In [102]:
! python ganspace.py --obj cat --comp_id 45 --scalar 60 --layers 5,7 --ckpt weights/my_full.pth --fixed_z z.pth --save_dir output/ganspace_eye_standing_cat

In [93]:
! python ganspace.py --obj cat --comp_id 2 --scalar 10 --layers 2,7 --ckpt weights/0.pth --fixed_z z.pth --save_dir output/ganspace_eye_standing_cat

__Note__: запоминаем точку в латенте

In [112]:
import torch

z = torch.randn(5, 512)
torch.save(z, 'z.pth')

In [164]:
i = 0

In [196]:
z1 = torch.randn(5, 512)
z2 = torch.randn(5, 512)

__Note__: интерполяция в латенте, потом пихал в онлайн сервисы по склейке gif'ок, результаты в презентации. Рандомил до тех пор, пока все стартовые точки не являлись норм котами. Раза 4 пришлось перекидывать. 

In [200]:
i = 0


if j == 0:
    alpha = 1.
else:
    alpha = 0.95

while alpha >= 0:
    model1 = torch.load(m1)
    model2 = torch.load(m2)

    zz = z1 * alpha + z2 * (1 - alpha)
    torch.save(zz, 'zz.pth')

    !python generate.py --ckpt weights/4.pth --save_dir output/kekw --fixed_z zz.pth

    for _ in range(5):
        os.replace('output/kekw/00000{}.png'.format(_), "output/kekw/{}/{}.png".format(_, i))

    i += 1
    alpha -= 0.05



/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: 

__Note__: интерполяция в весовом пространстве

In [193]:
i = 0

for j in range(4):
    m1 = 'weights/{}.pth'.format(j)
    m2 = 'weights/{}.pth'.format(j + 1)
    
    if j == 0:
        alpha = 1.
    else:
        alpha = 0.9
    
    while alpha >= 0:
        model1 = torch.load(m1)
        model2 = torch.load(m2)
        
        for name in model1:
            model1[name] = model1[name] * alpha + model2[name] * (1 - alpha)
            
        torch.save(model1, 'cringe.pth')
        
        !python generate.py --ckpt cringe.pth --save_dir output/samples_standing_cat2 --fixed_z z.pth
        
        for _ in range(5):
            os.replace('output/samples_standing_cat2/00000{}.png'.format(_), "output/samples_standing_cat2/{}/{}.png".format(_, i))

        i += 1
        alpha -= 0.1



/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: 

__Note__: Deprecated версия, семплил для себя по одному батчу

In [190]:
m1 = 'weights/2.pth'
m2 = 'weights/3.pth'

alpha = 0.7
print(alpha)

model1 = torch.load(m1)
model2 = torch.load(m2)

for name in model1:
    model1[name] = model1[name] * alpha + model2[name] * (1 - alpha)
    
torch.save(model1, 'cringe.pth')
##############
!python generate.py --ckpt cringe.pth --save_dir output/samples_standing_cat2 --fixed_z z.pth
##############

import glob
import os

for _ in range(5):
    os.replace('output/samples_standing_cat2/00000{}.png'.format(_), "output/samples_standing_cat2/{}/{}.png".format(_, i))

i += 1

0.7
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(


__Note__: генерировал картинки для себя и для презентации

In [91]:
!python generate.py --ckpt weights/0.pth --save_dir output/0
!python generate.py --ckpt weights/1.pth --save_dir output/1
!python generate.py --ckpt weights/2.pth --save_dir output/2
!python generate.py --ckpt weights/3.pth --save_dir output/3
!python generate.py --ckpt weights/4.pth --save_dir output/4

/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: UserWarning: The parameter 'range' is deprecated since 0.12 and will be removed in 0.14. Please use 'value_range' instead.
  warnings.warn(
/home/lemondrok/anaconda3/envs/gan/lib/python3.9/site-packages/torchvision/utils.py:63: 

__Note__: уже психанул и передернул код из оф. репа, удалив свой набросок. Там был код из репа с даталоадером и простой цикл обучения гана с 2 ган лоссами (+ был вариант l1 регуляризации и град пена как в статье)

In [ ]:
# opt, parser = get_opt()
# opt.isTrain = True

dataloader_sketch, sampler_sketch = create_dataloader(
    opt.dataroot_sketch,
    opt.size,
    opt.batch,
    opt.sketch_channel
)

In [ ]:
import time
import torch
import torch.multiprocessing as mp

from options import get_opt, print_options
from eval import Evaluator
from util.visualizer import Visualizer
from training.gan_trainer import GANTrainer
from training.dataset import create_dataloader, yield_data


def training_loop():
    torch.backends.cudnn.benchmark = True

    opt, parser = get_opt()
    opt.isTrain = True

    # needs to switch to spawn mode to be compatible with evaluation
    if not opt.disable_eval:
        mp.set_start_method('spawn')

    # dataloader for user sketches
    dataloader_sketch, sampler_sketch = create_dataloader(opt.dataroot_sketch,
                                                          opt.size,
                                                          opt.batch,
                                                          opt.sketch_channel)
    # dataloader for image regularization
    if opt.dataroot_image is not None:
        dataloader_image, sampler_image = create_dataloader(opt.dataroot_image,
                                                            opt.size,
                                                            opt.batch)
        data_yield_image = yield_data(dataloader_image, sampler_image)

    trainer = GANTrainer(opt)

    print_options(parser, opt)
    trainer.gan_model.print_trainable_params()
    if not opt.disable_eval:
        evaluator = Evaluator(opt, trainer.get_gan_model())
    visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots

    # the total number of training iterations
    if opt.resume_iter is None:
        total_iters = 0
    else:
        total_iters = opt.resume_iter

    optimize_time = 0.1
    for epoch in range(opt.max_epoch):
        iter_data_time = time.time()    # timer for data loading per iteration
        epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

        for i, data_sketch in enumerate(dataloader_sketch):  # inner loop within one epoch
            if total_iters >= opt.max_iter:
                return

            # makes dictionary to store all inputs
            data = {}
            data['sketch'] = data_sketch
            if opt.dataroot_image is not None:
                data_image = next(data_yield_image)
                data['image'] = data_image

            # timer for data loading per iteration
            iter_start_time = time.time()
            if total_iters % opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time

            # timer for optimization per iteration
            optimize_start_time = time.time()
            trainer.train_one_step(data, total_iters)
            optimize_time = (time.time() - optimize_start_time) * 0.005 + 0.995 * optimize_time

            # print training losses and save logging information to the disk
            if total_iters % opt.print_freq == 0:
                losses = trainer.get_latest_losses()
                visualizer.print_current_errors(epoch, total_iters, losses, optimize_time, t_data)
                visualizer.plot_current_errors(losses, total_iters)

            # display images on wandb and save images to a HTML file
            if total_iters % opt.display_freq == 0:
                visuals = trainer.get_visuals()
                visualizer.display_current_results(visuals, epoch, total_iters)

            # cache our latest model every <save_latest_freq> iterations
            if total_iters % opt.save_freq == 0:
                print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
                print(opt.name)  # it's useful to occasionally show the experiment name on console
                trainer.save(total_iters)

            # evaluate the latest model
            if not opt.disable_eval and total_iters % opt.eval_freq == 0:
                metrics_start_time = time.time()
                metrics, best_so_far = evaluator.run_metrics(total_iters)
                metrics_time = time.time() - metrics_start_time

                visualizer.print_current_metrics(epoch, total_iters, metrics, metrics_time)
                visualizer.plot_current_errors(metrics, total_iters)

            total_iters += 1
            epoch_iter += 1
            iter_data_time = time.time()
